## 7.10 数组

数组是通过简单地通过将元素连续存储在存储器中来实现的。
关于数组维度的信息不会被存储。
这使得在C和C++中使用数组比在其他编程语言中使用更快，但也更不安全。
通过定义一个类似于具有边界检查的数组的容器类，这个安全问题可以被克服，如下例所示：

```cpp
// Example 7.15a. Array with bounds checking
template <typename T, unsigned int N> class SafeArray {
protected:
    T a[N]; // Array with N elements of type T
public:
    SafeArray() { // Constructor
        memset(a, 0, sizeof(a)); // Initialize to zero
    }
    int Size() { // Return the size of the array
        return N;
    }
    T & operator[] (unsigned int i) { // Safe [] array index operator
        if (i >= N) {
            // Index out of range. The next line provokes an error.
            // You may insert any other error reporting here:
            return *(T*)0; // Return a null reference to provoke error
        }
        // No error
        return a[i]; // Return reference to a[i]
    }
}
```

更多关于容器类的例子：[www.agner.org/optimize/cppexamples.zip](www.agner.org/optimize/cppexamples.zip).

上述模板类的数组将类型和大小指定为模板参数，对该类的使用，如下面的示例7.15b所示。
就像普通数组一样，使用方括号索引访问元素。
构造函数将所有元素设置为零。
你可以删除该`memset`行，如果：
- 不需要此初始化，
- 或者类型T是一个类，有默认构造函数来执行必要初始化。

编译器可能会报告`memset`已被弃用。
这是因为如果`size`参数错误，它可能会导致错误，但`memset`仍然是将数组设置为零的最快方法。
如果索引超出范围，`[]`操作符将检测到错误（请参阅第138页上的边界检查）。
当错误消息被引发时，会通过返回空引用这种非常规的方式。
当访问此空引用时，又会引发错误消息（在受保护的操作系统上），使用调试器很容易跟踪此错误。
你可以替换此行，通过任何其他形式的错误报告。
例如，在Windows中，你可以换成`FatalAppExitA(0,"Array index out of range");`，或更好的方法，或者使用你自己的错误信息函数。

以下示例说明如何使用`SafeArray`：
```cpp
// Example 7.15b
SafeArray <float, 100> list; // Make array of 100 floats
for (int i = 0; i < list.Size(); i++) { // Loop through array
    cout << list[i] << endl; // Output array element
}
```

由列表初始化的数组应该最好是静态的，如第27页所述。
可以使用`memset`将数组初始化为0：
```cpp
// Example 7.16
float list[100];
memset(list, 0, sizeof(list));
```

对于多维数组，其设计和访问应该便于最后一个索引更改（频率）最快：
```cpp
// Example 7.17
const int rows = 20, columns = 50;
float matrix[rows][columns];
int i, j; float x;
for (i = 0; i < rows; i++)
    for (j = 0; j < columns; j++)
        matrix[i][j] += x;
```

这确保了元素被顺序访问。
如果两个循环的对调，会使对内存访问没有顺序，导致数据缓存效率降低。

如果行以非连续顺序被索引，**除了第一维以外的所有维的大小可以最好是为2的幂**，以便使地址计算更高效：

```cpp
// Example 7.18
int FuncRow(int); int FuncCol(int);
const int rows = 20, columns = 32;
float matrix[rows][columns];
int i; float x;
for (i = 0; i < 100; i++)
    matrix[FuncRow(i)][FuncCol(i)] += x;
```

在这里，代码必须计算`(FuncRow(i)*columns + FuncCol(i)) * sizeof(float)`才能找到矩阵元素的地址。
当列数是2的幂时，在这种情况下列的乘法更快。
在上例前面的示例中，这不是问题，因为优化的编译器可以看到行被连续访问，并且可以通过将行的长度加上前一行的地址，来计算每行的地址。

该建议也适用于结构或类对象的数组。 如果以非顺序方式访问元素，则对象的大小（以字节为单位）应优选为2的幂。

**将列数设置为2的幂的建议并不总适用于比一级数据高速缓存更大，并且非顺序访问的数组，因为这可能会导致缓存争用。
有关此问题的讨论，请参阅第89页。**